# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 9 2022 12:48PM,252737,19,MSP216597,Methapharm-G,Released
1,Dec 9 2022 12:41PM,252736,12,HH-38713,Hush Hush,Executing
2,Dec 9 2022 12:41PM,252736,12,HH-38714,Hush Hush,Released
3,Dec 9 2022 12:37PM,252735,15,30012775,"Alliance Pharma, Inc.",Released
4,Dec 9 2022 12:37PM,252735,15,30012776,"Alliance Pharma, Inc.",Released
5,Dec 9 2022 12:37PM,252735,15,30012777,"Alliance Pharma, Inc.",Released
6,Dec 9 2022 12:37PM,252735,15,30012778,"Alliance Pharma, Inc.",Released
7,Dec 9 2022 12:29PM,252734,10,MTR-22-0275,"Senseonics, Incorporated",Released
8,Dec 9 2022 12:29PM,252734,10,MTR-22-0276,"Senseonics, Incorporated",Released
9,Dec 9 2022 12:05PM,252729,10,0086237339,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,252734,Released,2
52,252735,Released,4
53,252736,Executing,1
54,252736,Released,1
55,252737,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252729,NaN,NaN,16.0
252734,NaN,NaN,2.0
252735,NaN,NaN,4.0
252736,NaN,1.0,1.0
252737,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252636,0.0,0.0,1.0
252637,0.0,0.0,1.0
252642,0.0,0.0,7.0
252644,0.0,0.0,1.0
252646,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252636,0,0,1
252637,0,0,1
252642,0,0,7
252644,0,0,1
252646,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252636,0,0,1
1,252637,0,0,1
2,252642,0,0,7
3,252644,0,0,1
4,252646,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252636,,,1
1,252637,,,1
2,252642,,,7
3,252644,,,1
4,252646,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 9 2022 12:48PM,252737,19,Methapharm-G
1,Dec 9 2022 12:41PM,252736,12,Hush Hush
3,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc."
7,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated"
9,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation
25,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd"
39,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation
47,Dec 9 2022 11:54AM,252725,10,Bio-PRF
48,Dec 9 2022 11:54AM,252724,10,Bio-PRF
49,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 9 2022 12:48PM,252737,19,Methapharm-G,,,1
1,Dec 9 2022 12:41PM,252736,12,Hush Hush,,1,1
2,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",,,4
3,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",,,2
4,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,,,16
5,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",,12,2
6,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation,,,8
7,Dec 9 2022 11:54AM,252725,10,Bio-PRF,,,1
8,Dec 9 2022 11:54AM,252724,10,Bio-PRF,,,1
9,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:48PM,252737,19,Methapharm-G,1,,
1,Dec 9 2022 12:41PM,252736,12,Hush Hush,1,1,
2,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4,,
3,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",2,,
4,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16,,
5,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",2,12,
6,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation,8,,
7,Dec 9 2022 11:54AM,252725,10,Bio-PRF,1,,
8,Dec 9 2022 11:54AM,252724,10,Bio-PRF,1,,
9,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:48PM,252737,19,Methapharm-G,1,,
1,Dec 9 2022 12:41PM,252736,12,Hush Hush,1,1,
2,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4,,
3,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",2,,
4,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:48PM,252737,19,Methapharm-G,1.0,NaN,NaN
1,Dec 9 2022 12:41PM,252736,12,Hush Hush,1.0,1.0,NaN
2,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4.0,NaN,NaN
3,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",2.0,NaN,NaN
4,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:48PM,252737,19,Methapharm-G,1.0,0.0,0.0
1,Dec 9 2022 12:41PM,252736,12,Hush Hush,1.0,1.0,0.0
2,Dec 9 2022 12:37PM,252735,15,"Alliance Pharma, Inc.",4.0,0.0,0.0
3,Dec 9 2022 12:29PM,252734,10,"Senseonics, Incorporated",2.0,0.0,0.0
4,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5053947,101.0,5.0,1.0
12,252736,1.0,1.0,0.0
15,1010826,18.0,0.0,0.0
19,1768907,9.0,12.0,0.0
20,252653,0.0,1.0,0.0
21,4800639,19.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5053947,101.0,5.0,1.0
1,12,252736,1.0,1.0,0.0
2,15,1010826,18.0,0.0,0.0
3,19,1768907,9.0,12.0,0.0
4,20,252653,0.0,1.0,0.0
5,21,4800639,19.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,101.0,5.0,1.0
1,12,1.0,1.0,0.0
2,15,18.0,0.0,0.0
3,19,9.0,12.0,0.0
4,20,0.0,1.0,0.0
5,21,19.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,101.0
1,12,Released,1.0
2,15,Released,18.0
3,19,Released,9.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,1.0,0.0,12.0,1.0,0.0
Released,101.0,1.0,18.0,9.0,0.0,19.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,1.0,0.0,12.0,1.0,0.0
2,Released,101.0,1.0,18.0,9.0,0.0,19.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,1.0,0.0,12.0,1.0,0.0
2,Released,101.0,1.0,18.0,9.0,0.0,19.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()